In [21]:
import PyPDF2

with open('./AIPF Emergenetics Profile/Profile_Narrative_Watson_Chua.pdf', 'rb') as f:
    # creating a page object
    pdfReader = PyPDF2.PdfFileReader(f)
    pageObj = pdfReader.getPage(0)
    # extracting text from page
    text = pageObj.extractText()

In [22]:
text

'HOW YOU THINK : PERCENTAGESHow you Think: Percentiles How you Behave: Percentiles\nGeneral Population   0          10         20          30         40         50          60         70         80          90         100Quiet              Introspective                Reserved               Talkative             Gregarious\nPeacekeeping           Easygoing             Competitive             Forceful               Driving\nFocused           Firm           Adaptable          Accommodating          Welcomes ChangeStructural\nSocial\nConceptual\nExpressiveness\nAssertiveness 19\n 5\n 69CONCEPTUAL = 38% ANALYTICAL = 49%\nSTRUCTURAL = 10%Imaginative\nIntuitive about ideas\nVisionary\nEnjoys the unusual\nLearns by experimenting\nSOCIAL = 3%\nHOW YOU COMPARE TO THE GENERAL POPULATIONClear thinker\nLogical problem solver\nData driven\nRational\nLearns by mental analysis\nPractical thinker\nLikes guidelines\nCautious of new ideas\nPredictable\nLearns by doingRelational\nIntuitive about people\n

In [28]:
import re
def extract_content_from_profile(profile):
    attributes = ['CONCEPTUAL', 'ANALYTICAL', 'STRUCTURAL', 'SOCIAL']
    results = {}
    for attr in attributes:
        value = re.findall(re.escape(attr) + r' = (\d+)%', text)[0]
        results[attr] = value
        results['NAME'] = text.split('\n')[-4].split('-')[0].strip()
    return results



In [29]:
from glob import glob
all_profiles = []
filepaths = glob('./AIPF Emergenetics Profile/*.pdf')
for fp in filepaths:
    with open(fp, 'rb') as f:
        pdfReader = PyPDF2.PdfFileReader(f)
        pageObj = pdfReader.getPage(0)
        text = pageObj.extractText()
        if not text.startswith('HOW YOU THINK'):
            print(text)
            continue
        profile = extract_content_from_profile(text)
        all_profiles.append(profile)
all_profiles

Xref table not zero-indexed. ID numbers for objects will be corrected.


[{'CONCEPTUAL': '34',
  'NAME': 'LU LI',
  'ANALYTICAL': '54',
  'STRUCTURAL': '9',
  'SOCIAL': '3'},
 {'CONCEPTUAL': '31',
  'NAME': 'AMELIA LEE',
  'ANALYTICAL': '42',
  'STRUCTURAL': '24',
  'SOCIAL': '3'},
 {'CONCEPTUAL': '22',
  'NAME': 'WEIGUANG CHEN',
  'ANALYTICAL': '38',
  'STRUCTURAL': '35',
  'SOCIAL': '5'},
 {'CONCEPTUAL': '10',
  'NAME': 'JEFFREY LONAN',
  'ANALYTICAL': '33',
  'STRUCTURAL': '49',
  'SOCIAL': '8'},
 {'CONCEPTUAL': '34',
  'NAME': 'PEK KUAN NG',
  'ANALYTICAL': '37',
  'STRUCTURAL': '13',
  'SOCIAL': '16'},
 {'CONCEPTUAL': '7',
  'NAME': 'WENJIE BOON',
  'ANALYTICAL': '23',
  'STRUCTURAL': '31',
  'SOCIAL': '39'},
 {'CONCEPTUAL': '3',
  'NAME': 'SULAIMAN ALWIE',
  'ANALYTICAL': '39',
  'STRUCTURAL': '42',
  'SOCIAL': '15'},
 {'CONCEPTUAL': '15',
  'NAME': 'JING HAN',
  'ANALYTICAL': '36',
  'STRUCTURAL': '22',
  'SOCIAL': '28'},
 {'CONCEPTUAL': '38',
  'NAME': 'WATSON CHUA',
  'ANALYTICAL': '49',
  'STRUCTURAL': '10',
  'SOCIAL': '3'},
 {'CONCEPTUAL': '39',

In [36]:
import pandas as pd
df_profiles = pd.DataFrame(all_profiles)

In [38]:
df_profiles.loc[df_profiles['NAME']=='WATSON CHUA']

,CONCEPTUAL,NAME,ANALYTICAL,STRUCTURAL,SOCIAL
8,38,WATSON CHUA,49,10,3


In [39]:
df_profiles.to_csv('profile.csv', index=False)

## Google Auth

In [11]:
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://www.googleapis.com/auth/drive.readonly']

credentials = ServiceAccountCredentials.from_json_keyfile_name('gdrive-access-368306-1d07a7f84034.json', scope)

# https://developers.google.com/drive/api/v3/quickstart/python
service = build('drive', 'v3', credentials=credentials)

# Call the Drive v3 API
results = service.files().list(
    fields="*",corpora = 'drive',supportsAllDrives = True, driveId = "1uoapecGhphu5AF5Lqd5jYhQYGBFO2j6u", includeItemsFromAllDrives = True).execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))

HttpError: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files?fields=%2A&corpora=drive&supportsAllDrives=true&driveId=1uoapecGhphu5AF5Lqd5jYhQYGBFO2j6u&includeItemsFromAllDrives=true&alt=json returned "Shared drive not found: 1uoapecGhphu5AF5Lqd5jYhQYGBFO2j6u". Details: "[{'domain': 'global', 'reason': 'notFound', 'message': 'Shared drive not found: 1uoapecGhphu5AF5Lqd5jYhQYGBFO2j6u', 'locationType': 'parameter', 'location': 'driveId'}]">

In [7]:
res = service.drives()

In [10]:
res

In [1]:
# Textract output
import json
with open('./watson_textract.json', 'r') as f:
    data = json.load(f)

In [8]:
all_text = []
for block in data['Blocks']:
    # print(block.keys())
    if "Text" in block:
        all_text.append(block["Text"])
        # print(block["Text"])
text_str = '\n'.join(all_text)

In [9]:
text_str

'R\nEMERGENETICS\nI\nPROFILE\nWATSON CHUA - OCTOBER 31, 2022\nHOW YOU THINK: PERCENTAGES\nANALYTICAL = 49%\nCONCEPTUAL = 38%\nClear thinker\nImaginative\nLogical problem solver\nIntuitive about ideas\nData driven\nVisionary\nRational\nEnjoys the unusual\nLearns by mental analysis\nLearns by experimenting\nSTRUCTURAL = 10%\nSOCIAL = 3%\nPractical thinker\nRelational\nLikes guidelines\nIntuitive about people\nCautious of new ideas\nSocially aware\nPredictable\nEmpathic\nLearns by doing\nLearns from others\nHOW YOU COMPARE TO THE GENERAL POPULATION\nAnalytical\n91\nStructural\n19\nSocial\n5\nConceptual\n69\nExpressiveness\n43\nQuiet\nIntrospective\nReserved\nTalkative\nGregarious\nAssertiveness\n75\nPeacekeeping\nEasygoing\nCompetitive\nForceful\nDriving\nFlexibility\n5\nFocused\nFirm\nAdaptable\nAccommodating\nWelcomes Change\nGeneral Population\n0\n10\n20\n30\n40\n50\n60\n70\n80\n90\n100\nC\nEmergenetics, LLC, 1991, 2022\nGeil Browning, Ph.D. / Wendell Williams, Ph.D.\nR\nEMERGENETICS\n

In [12]:
import re
attributes = ['CONCEPTUAL', 'ANALYTICAL', 'STRUCTURAL', 'SOCIAL']
# results = {}
for attr in attributes:
    value = re.findall(re.escape(attr) + r' = (\d+)%', text_str)[0]
    print(value)

38
49
10
3


In [17]:
percentile_pattern = r"\nAnalytical\n(\d+)\nStructural\n(\d+)\nSocial\n(\d+)\nConceptual\n(\d+)\nExpressiveness\n(\d+)\nQuiet\nIntrospective\nReserved\nTalkative\nGregarious\nAssertiveness\n(\d+)\nPeacekeeping\nEasygoing\nCompetitive\nForceful\nDriving\nFlexibility\n(\d+)"
re.findall(percentile_pattern, text_str)[0]

('91', '19', '5', '69', '43', '75', '5')